In [2]:
#Import
import pandas as pd
import numpy as np
import matplotlib as plt
import optuna

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Exemple from Kaggle
# rfc = RandomForestClassifier(max_depth=7, min_samples_split=10, random_state=2023)
# 
# smote = SMOTE(random_state=2023)

# rfc_pipe_num = Pipeline([
#     ('scaler', RobustScaler()),
# ])

# # for all object columns
# rfc_pipe_cat = Pipeline([
#     ('onehot', OneHotEncoder(drop='first')),
# ])

# # transforming all columns
# rfc_transformer = ColumnTransformer([
#     ('pipe_num', rfc_pipe_num, numeric_cols),
#     ('pipe_cat', rfc_pipe_cat, one_hot_cols)
# ])

# # combine all pipeline
# rfc_pipe_combine = Pipeline([
#     ('transformer', rfc_transformer),
#     ('rfe', RFE(rfc)),
#     ('resampling', smote),
#     ('rfc', rfc)
# ])

In [3]:
df = pd.read_csv('cleaned_data.csv')
df

,MIS_Status,LoanNr_ChkDgt,State,Zip,UrbanRural,Bank,BankState,NAICS,NoEmp,NewExist,...,FranchiseCode,IsFranchise,Term,RevLineCr,LowDoc,DisbursementGross,ApprovalDate,ApprovalFY,GrAppv,SBA_Appv
0,1,1000014003,IN,47711,0,FIFTH THIRD BANK,OH,451120,4,2.0,...,1,1,84,0,1,60000.0,1997-02-28,1997,60000.0,48000.0
1,1,1000024006,IN,46526,0,1ST SOURCE BANK,IN,722410,2,2.0,...,1,1,60,0,1,40000.0,1997-02-28,1997,40000.0,32000.0
2,1,1000034009,IN,47401,0,GRANT COUNTY STATE BANK,IN,621210,7,1.0,...,1,1,180,0,0,287000.0,1997-02-28,1997,287000.0,215250.0
3,1,1000044001,OK,74012,0,1ST NATL BK & TR CO OF BROKEN,OK,0,2,1.0,...,1,1,60,0,1,35000.0,1997-02-28,1997,35000.0,28000.0
4,1,1000054004,FL,32801,0,FLORIDA BUS. DEVEL CORP,FL,0,14,1.0,...,1,1,240,0,0,229000.0,1997-02-28,1997,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,1,9995573004,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,6,1.0,...,1,1,60,0,0,70000.0,1997-02-27,1997,70000.0,56000.0
899160,1,9995603000,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,6,1.0,...,1,1,60,0,0,85000.0,1997-02-27,1997,85000.0,42500.0
899161,1,9995613003,CA,93455,0,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,26,1.0,...,1,1,108,0,0,300000.0,1997-02-27,1997,300000.0,225000.0
899162,0,9995973006,HI,96830,0,BANK OF HAWAII,HI,0,6,1.0,...,1,1,60,0,1,75000.0,1997-02-27,1997,75000.0,60000.0


In [74]:

## Test L1 regularization

features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NoEmp', 'NewExist', 'FranchiseCode',
            'CreateJob', 'RetainedJob', 'IsFranchise', 'Term', 'RevLineCr', 'LowDoc',
            'DisbursementGross', 'GrAppv']
target = 'MIS_Status'

# Séparation des features et de la target
X = df[features]
y = df[target]

cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']

label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
    label_encoders[col] = le

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)


/tmp/ipykernel_44244/2093778282.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
/tmp/ipykernel_44244/2093778282.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
/tmp/ipykernel_44244/2093778282.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [76]:
y_pred=clf.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

ROC AUC: 0.8763


In [3]:
# Définir les colonnes
features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NoEmp', 'NewExist', 'FranchiseCode',
            'CreateJob', 'RetainedJob', 'IsFranchise', 'Term', 'RevLineCr', 'LowDoc',
            'DisbursementGross', 'GrAppv']
target = 'MIS_Status'

# Séparation des features et de la target
X = df[features]
y = df[target]

# Identifier les types de variables
cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']
num_features = ['NoEmp', 'CreateJob', 'RetainedJob', 'Term', 'DisbursementGross', 'GrAppv']

# Transformation des colonnes
preprocessor = ColumnTransformer([
    ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), cat_features),  # Meilleur que LabelEncoder en pipeline
    ("num", StandardScaler(), ['GrAppv', 'Term'])  # Seulement GrAppv et Term standardisé
], remainder="passthrough")  # Les autres colonnes restent inchangées

# Pipeline avec RandomForest
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=20, max_depth=7, min_samples_split=10,random_state=42))
])

# Division en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédiction
y_pred = pipeline.predict(X_test)

In [92]:

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

ROC AUC: 0.7523


In [93]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.53      0.64     15782
           1       0.91      0.98      0.94     74135

    accuracy                           0.90     89917
   macro avg       0.86      0.75      0.79     89917
weighted avg       0.89      0.90      0.89     89917



In [80]:
X

,State,UrbanRural,Bank,BankState,NoEmp,NewExist,FranchiseCode,CreateJob,RetainedJob,IsFranchise,Term,RevLineCr,LowDoc,DisbursementGross,GrAppv
0,IN,0,FIFTH THIRD BANK,OH,4,2.0,1,0,0,1,84,0,1,60000.0,60000.0
1,IN,0,1ST SOURCE BANK,IN,2,2.0,1,0,0,1,60,0,1,40000.0,40000.0
2,IN,0,GRANT COUNTY STATE BANK,IN,7,1.0,1,0,0,1,180,0,0,287000.0,287000.0
3,OK,0,1ST NATL BK & TR CO OF BROKEN,OK,2,1.0,1,0,0,1,60,0,1,35000.0,35000.0
4,FL,0,FLORIDA BUS. DEVEL CORP,FL,14,1.0,1,7,7,1,240,0,0,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,OH,0,JPMORGAN CHASE BANK NATL ASSOC,IL,6,1.0,1,0,0,1,60,0,0,70000.0,70000.0
899160,OH,0,JPMORGAN CHASE BANK NATL ASSOC,IL,6,1.0,1,0,0,1,60,0,0,85000.0,85000.0
899161,CA,0,"RABOBANK, NATIONAL ASSOCIATION",CA,26,1.0,1,0,0,1,108,0,0,300000.0,300000.0
899162,HI,0,BANK OF HAWAII,HI,6,1.0,1,0,0,1,60,0,1,75000.0,75000.0


In [94]:
prediction = pd.DataFrame([['CA', 0, 'FIFTH THIRD BANK', 'CA', 3, 1.0, 1, 3, 3, 0, 100, 0, 1, 600000, 600000]], columns=features)

y_pred=pipeline.predict(prediction)
print('prediction:', y_pred[0])

prediction: 1


In [95]:
prediction2 = df.iloc[1,:]

y_pred=pipeline.predict(prediction)
print('prediction:', y_pred[0])


prediction: 1


In [96]:
prediction2 = df.iloc[899162, :]

y_pred=pipeline.predict(prediction)
print('prediction:', y_pred[0])

prediction: 1


In [97]:
df.iloc[899162, :]


MIS_Status                        0
LoanNr_ChkDgt            9995973006
State                            HI
Zip                           96830
UrbanRural                        0
Bank                 BANK OF HAWAII
BankState                        HI
NAICS                             0
NoEmp                             6
NewExist                        1.0
CreateJob                         0
RetainedJob                       0
FranchiseCode                     1
IsFranchise                       1
Term                             60
RevLineCr                         0
LowDoc                            1
DisbursementGross           75000.0
ApprovalDate             1997-02-27
ApprovalFY                     1997
GrAppv                      75000.0
SBA_Appv                    60000.0
Name: 899162, dtype: object

In [30]:
df

,MIS_Status,LoanNr_ChkDgt,State,Zip,UrbanRural,Bank,BankState,NAICS,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,IsFranchise,Term,RevLineCr,LowDoc,DisbursementGross,ApprovalDate,ApprovalFY,GrAppv,SBA_Appv
0,1,1000014003,IN,47711,0,FIFTH THIRD BANK,OH,451120,4,2.0,0,0,1,1,84,0,1,60000.0,1997-02-28,1997,60000.0,48000.0
1,1,1000024006,IN,46526,0,1ST SOURCE BANK,IN,722410,2,2.0,0,0,1,1,60,0,1,40000.0,1997-02-28,1997,40000.0,32000.0
2,1,1000034009,IN,47401,0,GRANT COUNTY STATE BANK,IN,621210,7,1.0,0,0,1,1,180,0,0,287000.0,1997-02-28,1997,287000.0,215250.0
3,1,1000044001,OK,74012,0,1ST NATL BK & TR CO OF BROKEN,OK,0,2,1.0,0,0,1,1,60,0,1,35000.0,1997-02-28,1997,35000.0,28000.0
4,1,1000054004,FL,32801,0,FLORIDA BUS. DEVEL CORP,FL,0,14,1.0,7,7,1,1,240,0,0,229000.0,1997-02-28,1997,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,1,9995573004,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,6,1.0,0,0,1,1,60,0,0,70000.0,1997-02-27,1997,70000.0,56000.0
899160,1,9995603000,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,6,1.0,0,0,1,1,60,0,0,85000.0,1997-02-27,1997,85000.0,42500.0
899161,1,9995613003,CA,93455,0,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,26,1.0,0,0,1,1,108,0,0,300000.0,1997-02-27,1997,300000.0,225000.0
899162,0,9995973006,HI,96830,0,BANK OF HAWAII,HI,0,6,1.0,0,0,1,1,60,0,1,75000.0,1997-02-27,1997,75000.0,60000.0


In [4]:


# Définition de l'espace de recherche des hyperparamètres
param_dist = {
    "classifier__n_estimators": np.arange(50, 500, 50),  # Nombre d'arbres
    "classifier__max_depth": [None] + list(np.arange(5, 30, 5)),  # Profondeur max
    "classifier__min_samples_split": np.arange(2, 20, 2),  # Min échantillons pour diviser un nœud
    "classifier__min_samples_leaf": np.arange(1, 10, 2),  # Min échantillons par feuille
    "classifier__max_features": ['sqrt', 'log2', None],  # Nombre de features testées par split
    "classifier__bootstrap": [True, False]  # Méthode de bootstrap
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_dist,
    n_iter=30,  # Nombre d'itérations de recherche
    scoring="roc_auc",  # Optimisation basée sur le ROC AUC
    cv=5,  # Cross-validation
    verbose=2, 
    n_jobs=-1,  # Utilise tous les cœurs
    random_state=42
)

# Lancer la recherche
random_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres :", random_search.best_params_)
print("Meilleur score ROC AUC :", random_search.best_score_)

# Meilleur modèle
best_model = random_search.best_estimator_

# Prédiction sur le test set
y_pred = best_model.predict(X_test)

# Évaluation finale

print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END classifier__bootstrap=True, classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=3, classifier__min_samples_split=10, classifier__n_estimators=450; total time=31.4min


KeyboardInterrupt: 

In [5]:

# Espace de recherche réduit pour gagner du temps
param_dist = {
    "classifier__n_estimators": np.arange(50, 300, 50),  # Moins de valeurs testées
    "classifier__max_depth": [None, 5, 10, 15],  # Moins de profondeurs
    "classifier__min_samples_split": [2, 5, 10],  # Valeurs clés
    "classifier__min_samples_leaf": [1, 3, 5],  # Moins de choix
    "classifier__max_features": ['sqrt', 'log2'],  # Évite `None`
    "classifier__bootstrap": [True]  # Fixé à True pour simplifier
}

# RandomizedSearchCV rapide
random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_dist,
    n_iter=10,  # Réduit le nombre d'itérations
    scoring="roc_auc",
    cv=3,  # Validation croisée plus rapide
    verbose=1, 
    n_jobs=-1,  
    random_state=42
)

# Lancer la recherche
random_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres :", random_search.best_params_)
print("Meilleur score ROC AUC :", random_search.best_score_)

# Meilleur modèle
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Évaluation finale
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Meilleurs paramètres : {'classifier__n_estimators': np.int64(150), 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 1, 'classifier__max_features': 'log2', 'classifier__max_depth': None, 'classifier__bootstrap': True}
Meilleur score ROC AUC : 0.9697735808025131
              precision    recall  f1-score   support

           0       0.89      0.76      0.82     15782
           1       0.95      0.98      0.96     74135

    accuracy                           0.94     89917
   macro avg       0.92      0.87      0.89     89917
weighted avg       0.94      0.94      0.94     89917



In [ ]:
# Fonction d'optimisation
def objective(trial):
    # Definition of hyperparamaters
    n_estimators = trial.suggest_int("n_estimators", 100, 300, step=20)
    max_depth = trial.suggest_categorical("max_depth", [None, 5, 10, 15])
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10, step=2)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5, step=2)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])
    
    
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=True,
            random_state=42
        ))
    ])
    
    
    pipeline.fit(X_train, y_train)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Prendre les probabilités pour ROC AUC
    score = roc_auc_score(y_test, y_pred_proba)
    
    return score  # Maximiser le score AUC

# Lancer l'optimisation
study = optuna.create_study(direction="maximize",study_name="RandomForest_study13", storage="sqlite:///optuna_study.db")
study.optimize(objective, n_trials=50)  # Limité à 10 essais pour aller vite

# Meilleurs paramètres et score
print("Meilleurs paramètres :", study.best_params)
print("Meilleur score ROC AUC :", study.best_value)

# Entraîner le modèle final avec les meilleurs paramètres trouvés
best_params = study.best_params
best_model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(**best_params, bootstrap=True, random_state=42))
])

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Évaluation finale
print(classification_report(y_test, y_pred))

[I 2025-02-12 19:55:02,367] A new study created in RDB with name: RandomForest_study13
[I 2025-02-12 19:58:54,125] Trial 0 finished with value: 0.9695569213388013 and parameters: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9695569213388013.
[I 2025-02-12 20:00:34,996] Trial 1 finished with value: 0.8883018536509835 and parameters: {'n_estimators': 200, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.9695569213388013.
[I 2025-02-12 20:03:31,896] Trial 2 finished with value: 0.9695066007644779 and parameters: {'n_estimators': 140, 'max_depth': None, 'min_samples_split': 8, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9695569213388013.
[I 2025-02-12 20:04:51,543] Trial 3 finished with value: 0.8888082525605139 and parameters: {'n_estimators': 160, 'max_depth': 5, 'min_samples_split': 4, 'mi

Meilleurs paramètres : {'n_estimators': 300, 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Meilleur score ROC AUC : 0.9710735330214976
              precision    recall  f1-score   support

           0       0.89      0.77      0.82     15782
           1       0.95      0.98      0.97     74135

    accuracy                           0.94     89917
   macro avg       0.92      0.87      0.89     89917
weighted avg       0.94      0.94      0.94     89917



In [4]:
#Test avec les meilleurs paramètres

# Définir les colonnes
features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NoEmp', 'NewExist', 'FranchiseCode',
            'CreateJob', 'RetainedJob', 'IsFranchise', 'Term', 'RevLineCr', 'LowDoc',
            'DisbursementGross', 'GrAppv']
target = 'MIS_Status'

# Séparation des features et de la target
X = df[features]
y = df[target]

# Identifier les types de variables
cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']
num_features = ['NoEmp', 'CreateJob', 'RetainedJob', 'Term', 'DisbursementGross', 'GrAppv']

# Transformation des colonnes
preprocessor = ColumnTransformer([
    ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), cat_features),  # Meilleur que LabelEncoder en pipeline
    ("num", StandardScaler(), ['GrAppv', 'Term'])  # Seulement GrAppv et Term standardisé
], remainder="passthrough")  # Les autres colonnes restent inchangées


# Pipeline avec RandomForest
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=300, min_samples_leaf=1, max_features='sqrt', max_depth=None, min_samples_split=4, random_state=42))
])

# Division en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédiction
y_pred = pipeline.predict(X_test)

In [5]:
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

ROC AUC: 0.8740


In [6]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.77      0.82     15782
           1       0.95      0.98      0.97     74135

    accuracy                           0.94     89917
   macro avg       0.92      0.87      0.89     89917
weighted avg       0.94      0.94      0.94     89917



In [9]:
random_forest_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=300,
        min_samples_leaf=1,
        max_features='sqrt',
        max_depth=None,
        min_samples_split=4,
        random_state=42
    ))
])

In [10]:
# Entraînement
random_forest_pipeline.fit(X_train, y_train)

/home/utilisateur/Documents/accord_pret/loan_project/.venv/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['State', 'UrbanRural',
                                                   'Bank', 'BankState',
                                                   'NewExist', 'FranchiseCode',
                                                   'IsFranchise', 'RevLineCr',
                                                   'LowDoc']),
                                                 ('num', StandardScaler(),
                                                  ['GrAppv', 'Term'])])),
                ('classifier',
                 RandomForestClassifier(min_samples_split=4, n_estimators=300,
                                        random_state=42))])

In [ ]:
# # Emballage dans un BaggingClassifier
# bagging_pipeline = Pipeline([
#     ("preprocessor", preprocessor),
#     ("bagging", BaggingClassifier(
#         base_estimator=RandomForestClassifier(
#             n_estimators=100,  # Un peu moins pour chaque modèle
#             min_samples_leaf=1,
#             max_features='sqrt',
#             max_depth=None,
#             min_samples_split=4,
#             random_state=42
#         ),
#         n_estimators=10,  # 10 sous-modèles RandomForest
#         max_samples=0.8,  # Chaque modèle voit 80% des données
#         max_features=0.8,  # Chaque modèle utilise 80% des features
#         bootstrap=True,
#         n_jobs=-1,
#         random_state=42
#     ))
# ])

#Potentiel modèle de bagging

In [ ]:

def objective(trial):
    # Definition of hyperparamaters
    n_estimators = trial.suggest_int("n_estimators", 100, 300, step=20)
    max_depth = trial.suggest_categorical("max_depth", [None, 5, 10, 15])
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10, step=2)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5, step=2)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])
    
    # Creation of the pipeline
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=True,
            random_state=42
        ))
    ])
    
    # Train the pipeline
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test) 
    score = roc_auc_score(y_test, y_pred)
    
    return score 

# Start the optimization
study = optuna.create_study(direction="maximize",study_name="RandomForest_study", storage="sqlite:///optuna_study.db")
study.optimize(objective, n_trials=50)  

# Meilleurs paramètres et score
print("Meilleurs paramètres :", study.best_params)
print("Meilleur score ROC AUC :", study.best_value)

# Entraîner le modèle final avec les meilleurs paramètres trouvés
best_params = study.best_params
best_model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(**best_params, bootstrap=True, random_state=42))
])

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Évaluation finale
print(classification_report(y_test, y_pred))

NameError: name 'optuna' is not defined